In [15]:
import pandas as pd
import logging
import psycopg2
from sqlalchemy import create_engine
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s:%(funcName)s:%(levelname)s:%(message)s')

# Define the PostgreSQL connection parameters
db_params = {
    'dbname': "InvoiceDB",
    'user': "postgres",
    'password': 12345,
    'host': 'localhost',  # Replace with your host
    'port': '5432',  # Replace with your port
}

# Establish a connection to the PostgreSQL database
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')

source_dim = r"C:\Users\syakir1937\Task Scheduler\Solar Billing\Billing Database\Aeon billing - Testing.xlsx"
sheet_client = "ClientDim"

file_names = ['Aeon billing - Testing', 'Bill Statement CTRM latest', 'Bill Statement HTS']
sheet_names = ["Database_1", "PaymentDim"]  # Replace with your sheet names

# Define the start and end dates
start_date = '2022-01-01'
end_date = '2023-12-31'

meter_dim_column_name = {
        'Month': 'Date',
        'Meter Serial No': 'MeterID',
        'Initial Meter Reading' : 'Initial_Meter_Reading',
        'Last Meter Reading' : 'Last_Meter_Reading'
    }

payment_dim_column_name = {
        'Payment Received (RM)': 'Payment_Received',
        'Payment Date' : 'Date',
        'Bank Reference No' : 'Bank_Reference_No'
    }

def extract_and_concat(file_names, sheet_name):
    dataframes = []
    for file_name in file_names:
        source_path = r"C:\Users\syakir1937\Task Scheduler\Solar Billing\Billing Database\{}.xlsx".format(file_name)
        dataframes.append(pd.read_excel(source_path, sheet_name=sheet_name))   
    return pd.concat(dataframes, ignore_index=True)

def meter_dim(meter_df):
    df_meter_dim = meter_df.rename(columns=meter_dim_column_name) 
    # display(df_meter_dim)
    logging.info(' meter_dim data successfully transformed')
    return df_meter_dim

def client_dim(source_dim, sheet_client):
    df_client_dim = pd.read_excel(source_dim, sheet_name=sheet_client, header=0)
    logging.info(' client_dim successfully transformed')
    return df_client_dim

def payment_dim(payment_df):
    df_payment_dim = payment_df.rename(columns=payment_dim_column_name) 
    df_payment_dim['Date'] = pd.to_datetime(df_payment_dim['Date'])
    df_payment_dim['Date'] = df_payment_dim['Date'].dt.date
    display(df_payment_dim)
    logging.info(' payment_dim data successfully transformed')
    return df_payment_dim

def create_date_table(start_date, end_date):
    # Create a date range from start_date to end_date
    date_range = pd.date_range(start_date, end_date, freq='D')    
    # Create a DataFrame with the date range
    date_df = pd.DataFrame({'Date': date_range})   
    # Extract year and month into separate columns
    date_df['Year'] = date_df['Date'].dt.year
    date_df['Month'] = date_df['Date'].dt.month
    date_df['DateID'] = date_df['Date'].dt.strftime('%Y%m') 
    logging.info(' date_dim data successfully transformed')  
    return date_df

def load(df_meter_dim, df_client_dim, df_payment_dim, date_table):
    df_meter_dim.to_sql('DimMeter', engine, if_exists='replace', index=False)
    df_client_dim.to_sql('DimClient', engine, if_exists='replace', index=False)
    df_payment_dim.to_sql('DimPayment', engine, if_exists='replace', index=False)
    date_table.to_sql('DimDate', engine, if_exists='replace', index=False)

# Close the database connection
engine.dispose()

def get_data_excel():
    meter_df = extract_and_concat(file_names, "Database_1")
    payment_df = extract_and_concat(file_names, "PaymentDim")
    df_meter_dim = meter_dim(meter_df)
    df_client_dim = client_dim(source_dim, sheet_client)
    df_payment_dim = payment_dim(payment_df)
    date_table = create_date_table(start_date, end_date)
    load(df_meter_dim, df_client_dim, df_payment_dim, date_table)
        
if __name__ == '__main__':
    get_data_excel()

2023-10-22 13:18:12,819:meter_dim:INFO: meter_dim data successfully transformed
2023-10-22 13:18:12,918:client_dim:INFO: client_dim successfully transformed


,ClientID,Payment_Type,Payment_Received,Date,Bank_Reference_No
0,8,Bank Transfer,56588.3800,2022-07-14,20001361440008202206
1,8,Bank Transfer,55383.3900,2022-08-19,20001373700008202207
2,8,Bank Transfer,62032.4000,2022-09-14,20001376368202208
3,8,Bank Transfer,56875.7400,2022-10-14,20001384940008202209
4,8,Bank Transfer,59474.3500,2022-11-14,100002105672494
5,8,Bank Transfer,51069.0500,2022-12-14,2000140112
6,8,Bank Transfer,50191.7500,2023-03-14,100002105813223
7,8,Bank Transfer,52409.1500,2023-03-14,100002105891180
8,8,Bank Transfer,51474.6000,2023-03-14,100002105944725
9,3,Bank Transfer,78410.4000,2023-01-16,MYIT230111533543


2023-10-22 13:18:12,935:payment_dim:INFO: payment_dim data successfully transformed
2023-10-22 13:18:12,944:create_date_table:INFO: date_dim data successfully transformed
